1\. Write a function that converts number representation (bin<->dec<->hex)

In [35]:

def convertitore(numero,base):
        if base == "decimale":
            a=""
            b=""
            hexa=numero
            while numero != 0:
                a+=str(numero%2) 
                numero=numero//2
            print("Il numero in base binaria è ","0b"+a[::-1])
            while hexa != 0:
                if hexa%16 <= 9 :
                    b+=str(hexa%16)
                    hexa=hexa//16
                if hexa%16 == 10 :
                    b+="A"
                    hexa=hexa//16
                if hexa%16 == 11 :
                    b+="B"
                    hexa=hexa//16
                if hexa%16 == 12 :
                    b+="C"
                    hexa=hexa//16
                if hexa%16 == 13 :
                    b+="D"
                    hexa=hexa//16
                if hexa%16 == 14 :
                    b+="E"
                    hexa=hexa//16                
                if hexa%16 == 15 :
                    b+="F"
                    hexa=hexa//16                    
            print("Il numero in base esadecimale è ","0b"+b[::-1])
               
        if base == "binaria" :
            a=[]
            for i in range(len(str(numero))):
                val=int(str(numero)[len(str(numero))-1-i])
                a.append(pow(2,i)*val)
            print("Il numero in base decimale è", str(sum(a)))  

            convertitore(int(str(sum(a))),"decimale")

        if base == "esadecimale" :

            hbin={'0':'0000','1':'0001','2':'0010','3':'0011','4':'0100','5':'0101','6':'0110','7':'0111','8':'1000','9':'1001','A':'1010','B':'1011','C':'1100','D':'1101','E':'1110','F':'1111'}
            a=[]
            for i in range(len(str(numero))):
                a.append(hbin[str(numero)[i]])
            j="".join(a)
            jl=list(j)
            i=0

            convertitore(int("".join(jl)),"binaria")

convertitore("6A","esadecimale")
        
        
    

Il numero in base decimale è 106
Il numero in base binaria è  0b1101010
Il numero in base esadecimale è  0b6A


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [36]:
word='00101001010001110110100101101010'

s=int(word[0])
expo=0
mant=1
bias=127

for i in range(1,len(word)):
    if i<9:
        expo+=(2**((8-i)))*int(word[i])
    else:
        mant+=(2**(-(i-8)))*int(word[i])


        
print(((-1)**(s))*mant*(2**(expo-bias)))

4.427830850453965e-14


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [37]:
und=1
over=1

while True:
    try:
        over*=2
        (over/2)*2
    except:
        break

while True:
    try:
        und/=2
        1/(und/2)
    except:
        break

print(over,und)

359538626972463181545861038157804946723595395788461314546860162315465351611001926265416954644815072042240227759742786715317579537628833244985694861278948248755535786849730970552604439202492188238906165904170011537676301364684925762947826221081654474326701021369172596479894491876959432609670712659248448274432 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [38]:
a=5
i=1

while True:
    b=5+(2**(-i))
    i+=1
    if b==a:
        break

print(2**(-i+1))

4.440892098500626e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [4]:
import math

a=0.001
b=1000
c=0.001

xpiu=(-b+pow((b*b-4*a*c),1/2))/(2*a)
xmeno=(-b-pow((b*b-4*a*c),1/2))/(2*a)
p1=-b+pow(b*b-4*a*c,1/2)

print(xpiu,p1,xmeno)

xpiu1=((-b+pow(b*b-4*a*c,1/2))*(-b-pow(b*b-4*a*c,1/2)))/((2*a)*(-b-pow(b*b-4*a*c,1/2)))
xmeno1=((-b-pow((b*b-4*a*c),1/2))*(-b+pow(b*b-4*a*c,1/2)))/((2*a)*(-b+pow(b*b-4*a*c,1/2)))

print(xpiu1,xmeno1)

-9.999894245993346e-07 -1.999978849198669e-09 -999999.999999
-9.999894245993346e-07 -999999.9999990001


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [40]:
x=1
delt=0
tup=(2,4,6,8,10,12,14)
print(delt)
def fx(r):
    return(r*(r-1))

def derivata(n,delta):
    return((fx(n+delta)-fx(n))/delta)

for i in tup:
    d=1.0*(10**(-i))
    print('Valore delta=10^-',i,':', derivata(x,d))

print('Valore esatto:', (x-1)+x)

0
Valore delta=10^- 2 : 1.010000000000001
Valore delta=10^- 4 : 1.0000999999998899
Valore delta=10^- 6 : 1.0000009999177333
Valore delta=10^- 8 : 1.0000000039225287
Valore delta=10^- 10 : 1.000000082840371
Valore delta=10^- 12 : 1.0000889005833413
Valore delta=10^- 14 : 0.9992007221626509
Valore esatto: 1


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [18]:


def fz(x):
    return pow((1-x*x),1/2)

def integral(f,xmin,xmax,N):
    somma=0
    numba=0
        for i in range(N):
            somma+=((xmax-xmin)/N)*f(xmin+i*((xmax-xmin)/N))    
        return somma
        

I=(integral(fz,-1,1,100000))
print(I)
print("Differenza reale e algoritmo", (math.pi/2 - I))


%timeit integral(fz,-1,1,200000)

1.5707962742034223
Differenza reale e algoritmo 5.2591474286600715e-08
412 ns ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)
